In [19]:
class Station:
    import notebook_importer
    import numpy as np
    from Logger      import Logger
    nb_d_instance  = 0
    station_logger = Logger("./stations.csv")
    output_dir     = "./Stations/"

    def __init__(self, position):
        import notebook_importer
        import random
        from Velo   import Velo
        from Logger import Logger
        import os
        import numpy as np
        Station.nb_d_instance +=1
        
        NB_BORNES_POSSIBLES = [5,10,15,20,30]
        self.position       = position
        self.nb_bornes      = random.choice(NB_BORNES_POSSIBLES)
        self.en_panne       = False
        self.velos          = [Velo() for i in range(self.nb_bornes-3)]
        self.id             = Station.nb_d_instance
        
        self.nb_usage_pour_degradation   = (np.random.normal(50, 5) + self.nb_bornes) * (max(1, self.position.longitude) * np.random.normal(1, 0.05))
        self.degradation                 = self.nb_bornes / np.random.normal(500, 50) 
        self.limite_pour_etre_en_panne   = min(self.position.lattitude / 100, 0.2)
        self.nb_usage            = 0
        self.performance         = 1
        
        self.logger      = Logger(os.path.join(Station.output_dir, "log_stations_%s.csv"%self.id))
        log_format       = "timestamp,station_id,cycliste_id,velo_id,velo_performance"
        performance_log  = "timestamp,station_id,performance"
        self.logger.log(log_format)
        self.logger.log(   log_format)
        self.logger.log(performance_log)
        
    def reparer(self, timestamp):
        self.flush_logs()
        self.nb_usage            = 0
        self.performance         = 1    
        self.en_panne            = False
        self.logger.log(self.get_performance_log(timestamp))
        
        [velo.reparer for velo in self.velos]
        self.logger.log("reparee")

    def __str__(self):
        to_print = "Station n° {nom} : {position}, {nb_bornes} bornes, {nb_velos_dispo} velos"

        return to_print.format(nom=self.id,
                               position=self.position,
                              nb_bornes=self.nb_bornes,
                              nb_velos_dispo=len(self.velos))
    def flush_logs(self):
        log_objetcs = [self.logger]
        for log in log_objetcs:
            log.write_to_disk()

    def nb_de_velos(self):
        return len(self.velos)

    def has_empty_place(self):
        return len(self.velos) < self.nb_bornes

    def has_velo(self):
        return 0 < len(self.velos)

    def log(self, message):
        message = "station %s, %s"%(self.id, message)
        self.logger.log(message)
        
    def get_log_message(self, timestamp, cycliste, velo):
        return ",".join([str(timestamp), str(self.id), str(cycliste.id), str(velo.id), str(velo.performance)])
    
    def get_performance_log(self, timestamp):
        return ",".join([str(timestamp), str(self.id), str(self.performance)])
    
    def update_performance(self, timestamp):
        import random
        if self.nb_usage_pour_degradation < self.nb_usage :
            self.performance = self.performance - (self.degradation * random.choice([0]*5 + [1]))
        if self.performance < self.limite_pour_etre_en_panne:
            self.en_panne = True
        self.logger.log(self.get_performance_log(timestamp))
        
        
    def deposer_un_velo(self, timestamp, cycliste, velo):
        if self.en_panne:
            self.logger.log("en panne")
            return False
            
        if self.has_empty_place():
            message = self.get_log_message(timestamp, cycliste, velo)
            self.logger.log(message)
            velo.update_performance(timestamp)
            self.velos.append(velo)
            self.update_performance(timestamp)
            self.nb_usage +=1
            return True
        else:
            self.logger.log("complete")
            return False
    def prendre_un_velo(self, timestamp, cycliste):
        if self.en_panne:
            self.logger.log("en panne")
            return False
        
        if self.has_velo():
            import random
            velo    = self.velos.pop(random.randint(0, len(self.velos)-1))
            message = self.get_log_message(timestamp, cycliste, velo)
            self.logger.log(message)
            self.update_performance(timestamp)
            self.nb_usage +=1
            return velo
        else:
            self.logger.log("vide.")
            return False

    def nb_velo_under_perf(self, seuil_de_performance=0.5):
        test = lambda velo: velo.performance < seuil_de_performance
        return len(list(filter(test, self.velos)))
